In [1]:
from faiss_module import make_db, make_fewshot_db, get_embedding
import pandas as pd

train_df = pd.read_csv('train.csv')
# fewshot_db = make_fewshot_db(train_df)
train_db = make_db(train_df,'./train_faiss_db')
train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8,'k':3})

test_df = pd.read_csv('test.csv')
test_db = make_db(test_df,'./test_faiss_db')


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [11]:
import warnings

# 특정 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore", message="No relevant docs were retrieved using the relevance score threshold")


rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0}

for i , entry in enumerate(train_df.to_dict(orient='records')):
    # print(entry['Question'])
    # print(entry['Source_path'])
    train_retriever = train_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
    rag = train_retriever.invoke(entry['Question'])

    num_rag = len(rag)
    if num_rag ==0 :
        print(i)
        print(entry['Question'])
        print(entry['Source_path'])
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

{'0': 0, '1': 6, '2': 490, '3': 0, '4': 0, '5': 0}


In [8]:
rag_counter = {'0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0}
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.77,'k':2})
for i , entry in enumerate(test_df.to_dict(orient='records')):
    num_rag = len(test_retriver.invoke(entry['Question']))
    if num_rag ==0 :
        rag_counter['0'] += 1
    if num_rag == 1:
        rag_counter['1'] += 1
    elif num_rag == 2:
        rag_counter['2'] += 1
    elif num_rag == 3:
        rag_counter['3'] += 1
    elif num_rag == 4:
        rag_counter['4'] += 1
    elif num_rag == 5:
        rag_counter['5'] += 1
print(rag_counter)

{'0': 0, '1': 4, '2': 94, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0}


In [9]:
i = 67
querry = test_df.iloc[i]['Question']
test_retriver = test_db.as_retriever(search_kwargs={'k': 3})
print("Q: ",querry)
len(test_retriver.invoke(querry))
# print(doc_list[0].page_content)
# print(doc_list[0].metadata)

Q:  2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


3

In [3]:
test_retriver = test_db.as_retriever(search_kwargs={'search_tpye':'ngram','k': 3})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  2015년도 평가 이전에는 평가 결과 미흡 등급 비율을 강제 배정하고 해당 단위사업에 대한 일률적 삭감을 실\n시→단위 사업 내 성과가 우수한 세부사업이 삭감되는 일도 발생하고, 사업 성격상 구조조정이 어려운 사업\n도평가를 통한 환류가 어려웠음\n-  2016년도 환류 개선을 통해 평가 대상 사업 전체 예산의 1% 이내에서 부처에서 자율적으로 세출구조조정을 \n마련하도록 하고우수등급사업도지출구조조정이 가능하게하는한편,구조조정이 어려운사업에대해서는\n성과관리개선대책을 제출하게 하였으며 현재도 이 틀에서 환류 수행\n‣ 평가는지속적으로 간소화되면서, 환류로 인한 재정관리 합리성을 강화하는 데 초점\n-  2015년까지는 평가지표를 11개로 하였으나, 이후에는 4개로 단순화\n-  2018년도 제도 개선으로 각 부처에서 지표와 배점 기준을 자율적으로 하도록 하여, 부처의 자율적인 재정사\n업성과관리 확대\n-  제도개선으로 성과관리체계 단순화, 예산과 성과관리의 기계적 연동 폐지, 합리적 재정사업 구조조정 등으'),
 Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 8}, page_content='박정수.(2020). dBr ain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.\nOECD.(2021). Go vernment at a Glanc e 2021.\n이 보고서는 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.'),
 Document(metadata={'source': './test_source/「F

In [4]:
test_retriver = test_db.as_retriever(search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  2015년도 평가 이전에는 평가 결과 미흡 등급 비율을 강제 배정하고 해당 단위사업에 대한 일률적 삭감을 실\n시→단위 사업 내 성과가 우수한 세부사업이 삭감되는 일도 발생하고, 사업 성격상 구조조정이 어려운 사업\n도평가를 통한 환류가 어려웠음\n-  2016년도 환류 개선을 통해 평가 대상 사업 전체 예산의 1% 이내에서 부처에서 자율적으로 세출구조조정을 \n마련하도록 하고우수등급사업도지출구조조정이 가능하게하는한편,구조조정이 어려운사업에대해서는\n성과관리개선대책을 제출하게 하였으며 현재도 이 틀에서 환류 수행\n‣ 평가는지속적으로 간소화되면서, 환류로 인한 재정관리 합리성을 강화하는 데 초점\n-  2015년까지는 평가지표를 11개로 하였으나, 이후에는 4개로 단순화\n-  2018년도 제도 개선으로 각 부처에서 지표와 배점 기준을 자율적으로 하도록 하여, 부처의 자율적인 재정사\n업성과관리 확대\n-  제도개선으로 성과관리체계 단순화, 예산과 성과관리의 기계적 연동 폐지, 합리적 재정사업 구조조정 등으'),
 Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 8}, page_content='박정수.(2020). dBr ain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.\nOECD.(2021). Go vernment at a Glanc e 2021.\n이 보고서는 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.'),
 Document(metadata={'source': './test_source/「F

In [5]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, 'score_threshold': 0.8, "k":3})
print(querry)
test_retriver.invoke(querry)

2016년 재정성과관리제도의 환류 개선사항은 무엇인가?


[Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  2015년도 평가 이전에는 평가 결과 미흡 등급 비율을 강제 배정하고 해당 단위사업에 대한 일률적 삭감을 실\n시→단위 사업 내 성과가 우수한 세부사업이 삭감되는 일도 발생하고, 사업 성격상 구조조정이 어려운 사업\n도평가를 통한 환류가 어려웠음\n-  2016년도 환류 개선을 통해 평가 대상 사업 전체 예산의 1% 이내에서 부처에서 자율적으로 세출구조조정을 \n마련하도록 하고우수등급사업도지출구조조정이 가능하게하는한편,구조조정이 어려운사업에대해서는\n성과관리개선대책을 제출하게 하였으며 현재도 이 틀에서 환류 수행\n‣ 평가는지속적으로 간소화되면서, 환류로 인한 재정관리 합리성을 강화하는 데 초점\n-  2015년까지는 평가지표를 11개로 하였으나, 이후에는 4개로 단순화\n-  2018년도 제도 개선으로 각 부처에서 지표와 배점 기준을 자율적으로 하도록 하여, 부처의 자율적인 재정사\n업성과관리 확대\n-  제도개선으로 성과관리체계 단순화, 예산과 성과관리의 기계적 연동 폐지, 합리적 재정사업 구조조정 등으'),
 Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 8}, page_content='박정수.(2020). dBr ain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.\nOECD.(2021). Go vernment at a Glanc e 2021.\n이 보고서는 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.'),
 Document(metadata={'source': './test_source/「F

In [6]:
test_db.similarity_search_with_relevance_scores(querry, k=10, score_threshold=0.8,filter={'source':test_df.iloc[i]['Source_path']})


[(Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  2015년도 평가 이전에는 평가 결과 미흡 등급 비율을 강제 배정하고 해당 단위사업에 대한 일률적 삭감을 실\n시→단위 사업 내 성과가 우수한 세부사업이 삭감되는 일도 발생하고, 사업 성격상 구조조정이 어려운 사업\n도평가를 통한 환류가 어려웠음\n-  2016년도 환류 개선을 통해 평가 대상 사업 전체 예산의 1% 이내에서 부처에서 자율적으로 세출구조조정을 \n마련하도록 하고우수등급사업도지출구조조정이 가능하게하는한편,구조조정이 어려운사업에대해서는\n성과관리개선대책을 제출하게 하였으며 현재도 이 틀에서 환류 수행\n‣ 평가는지속적으로 간소화되면서, 환류로 인한 재정관리 합리성을 강화하는 데 초점\n-  2015년까지는 평가지표를 11개로 하였으나, 이후에는 4개로 단순화\n-  2018년도 제도 개선으로 각 부처에서 지표와 배점 기준을 자율적으로 하도록 하여, 부처의 자율적인 재정사\n업성과관리 확대\n-  제도개선으로 성과관리체계 단순화, 예산과 성과관리의 기계적 연동 폐지, 합리적 재정사업 구조조정 등으'),
  0.8810049059910099),
 (Document(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 8}, page_content='박정수.(2020). dBr ain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.\nOECD.(2021). Go vernment at a Glanc e 2021.\n이 보고서는 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.'),
  0.8563747934834256),

In [ ]:
test_retriver = test_db.as_retriever(search_kwargs={'filter': {'source':test_df.iloc[i]['Source_path']}, "k":3})
print(querry)
test_retriver.invoke(querry)

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings = get_embedding()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.80)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=test_retriver
)
compression_retriever.invoke(querry)

[_DocumentWithState(metadata={'source': './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf', 'page': 6}, page_content='-  2015년도 평가 이전에는 평가 결과 미흡 등급 비율을 강제 배정하고 해당 단위사업에 대한 일률적 삭감을 실\n시→단위 사업 내 성과가 우수한 세부사업이 삭감되는 일도 발생하고, 사업 성격상 구조조정이 어려운 사업\n도평가를 통한 환류가 어려웠음\n-  2016년도 환류 개선을 통해 평가 대상 사업 전체 예산의 1% 이내에서 부처에서 자율적으로 세출구조조정을 \n마련하도록 하고우수등급사업도지출구조조정이 가능하게하는한편,구조조정이 어려운사업에대해서는\n성과관리개선대책을 제출하게 하였으며 현재도 이 틀에서 환류 수행\n‣ 평가는지속적으로 간소화되면서, 환류로 인한 재정관리 합리성을 강화하는 데 초점\n-  2015년까지는 평가지표를 11개로 하였으나, 이후에는 4개로 단순화\n-  2018년도 제도 개선으로 각 부처에서 지표와 배점 기준을 자율적으로 하도록 하여, 부처의 자율적인 재정사\n업성과관리 확대\n-  제도개선으로 성과관리체계 단순화, 예산과 성과관리의 기계적 연동 폐지, 합리적 재정사업 구조조정 등으', state={'embedded_doc': [0.046563584357500076, -0.0176229327917099, -0.021066928282380104, -0.04302213340997696, 0.05149956792593002, 0.008888784795999527, 0.03424005210399628, 0.08838507533073425, 0.0950552374124527, 0.0616384893655777, 0.044455356895923615, 0.0442797876894474, 0.0502285435795784, -0.013452444225549698, -0.04037046059966087, 0.04409484565258026